This notebook change the length of all wav files preprocessed to 22kHz 

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np 
from analysis import quantize
import os 

In [2]:
filepath = '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/'
listfiles = os.listdir(filepath)
listnpz = []
for c in listfiles:
    if c[-3:] == 'npz':
        listnpz.append(os.path.join(filepath,c))

In [7]:
listnpz


['/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess3.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess5_2.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess5_1.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess2.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess4_1.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess6.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess4_3.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_sess4_2.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_perf1.npz',
 '/home/brain/Desktop/BrainSongs/audiosummary_private/christopherocher/caroleeg_perf2.npz']

In [3]:
import soundfile as sf
def load(filepath):
    sound, sr = sf.read(filepath,dtype ='float32')   
    return sound,sr

In [4]:

def cut(filepath,addnoise = True):
    resultdir = os.path.join('data_clean')
    os.makedirs(resultdir,exist_ok=True)
    # audio 22kHz, mono
    audiopath = '/home/brain/Desktop/BrainSongs/audiosummary_private/audio22k/'
    for curnpz in listnpz:
        focus = np.load(curnpz)['focus']
        
        subjtime = np.load(curnpz)['subjtime']
        #Quantization
        focus_q = quantize(focus,res=1,delaytime=0) 
        subt_q = quantize(subjtime,res=1,delaytime=1)

        maxtime = min(focus_q.shape[0],subt_q.shape[0])

        if addnoise:
            focus_q[:maxtime,1] += np.random.rand(maxtime)
            subt_q[:maxtime,1] += np.random.rand(maxtime)
        
        ## name of audiofile
        name = curnpz.split('/')[7][:-4] 
        audio,sr = load("{}{}_.wav".format(audiopath,name))
        print("Maxtime : {}s, audio.shape : {}s".format(maxtime,audio.shape[0]/sr))
        if audio.shape[0] > maxtime*sr:
            audio = audio[:maxtime*sr]        
        #save
        sf.write('{}/{}.wav'.format(resultdir,name), audio, sr)
        
        print("audio ok, new shape : {}s".format(audio.shape[0]/sr))

In [41]:
#quantize and save data
cut(filepath)

Maxtime : 1701s, audio.shape : 1766.7273015873016s
audio ok, new shape : 1701.0s
npz ok
Maxtime : 895s, audio.shape : 951.2727437641723s
audio ok, new shape : 895.0s
npz ok
Maxtime : 602s, audio.shape : 698.1818594104309s
audio ok, new shape : 602.0s
npz ok
Maxtime : 907s, audio.shape : 977.4545578231292s
audio ok, new shape : 907.0s
npz ok
Maxtime : 543s, audio.shape : 558.5454875283447s
audio ok, new shape : 543.0s
npz ok
Maxtime : 1724s, audio.shape : 1741.0909297052153s
audio ok, new shape : 1724.0s
npz ok
Maxtime : 453s, audio.shape : 558.5454875283447s
audio ok, new shape : 453.0s
npz ok
Maxtime : 594s, audio.shape : 558.5454875283447s
audio ok, new shape : 558.5454875283447s
npz ok
Maxtime : 789s, audio.shape : 855.2727437641723s
audio ok, new shape : 789.0s
npz ok
Maxtime : 1048s, audio.shape : 1077.818185941043s
audio ok, new shape : 1048.0s
npz ok


In [5]:
def get_values():
    to_time = {}    
    with open('/home/brain/Desktop/BrainSongs/audiosummary_private/soundnet/relation_layer_seconds.txt', 'r') as reader:
        for i in reader:
            key,m,b = i.split()
            if key != 'name_layer':
                to_time[key] = [float(m),float(b)]
    return to_time

In [10]:
from graphics import *
#change resolution to conv7 and align data from cluster
def change_to_res_audio(name_layer,addnoise = True,cluster_path = './results_13_08/'):
    resultdir = os.path.join('data_clean_')
    os.makedirs(resultdir,exist_ok=True)
  
    #find resolution
    cursr = 22050
    m,b = get_values()[name_layer] #get slope (m) and interception (b) for a given layer
    res = (1/m) #separation in time
    
    #path to data clusters
    
   
    for curnpz in listnpz:
        focus = np.load(curnpz)['focus']
        
        subjtime = np.load(curnpz)['subjtime']
        #Quantization
        focus_q = quantize(focus,res=res,delaytime=0) 
        subt_q = quantize(subjtime,res=res,delaytime=1)

        #find the min include vector of clustering
        ## name of audiofile
        name = curnpz.split('/')[7][:-4]  
        #labels_audio
        labels = np.load("{}{}/features_conv_v27.npz".format(cluster_path,name))['labels']
        print(focus_q.shape[0],subt_q.shape[0],labels.shape[0])
        maxtime = min(focus_q.shape[0],subt_q.shape[0],labels.shape[0])
        
        labels_audio = labels[:maxtime,1]
        
        if addnoise:
            focus_q[:maxtime,1] += np.random.rand(maxtime)
            subt_q[:maxtime,1] += np.random.rand(maxtime)
        
        #make cluster
        focus = focus_q[:maxtime,1]
        subjtime  = subt_q[:maxtime,1]
        
        matrix = get_matrix(focus,subjtime,curnpz)
        labels, cal = make_cluster(matrix,True)
        
        
        print("File {}, length {}, res {:.2f}".format(name,maxtime,res))
        np.savez_compressed("{}/{}.npz".format(resultdir,name),
                            focus = focus,
                            subjtime = subjtime,
                            labels = labels,
                            cal = cal,
                           labels_audio = labels_audio)
        print("npz saved in data clean : focus, subjtime and labels ")

In [11]:
change_to_res_audio('conv7')

 11%|█         | 1/9 [00:00<00:01,  7.12it/s]

1171 1139 1146
(1139, 2)


  0%|          | 0/9 [00:00<?, ?it/s]

Best number of cluster is 2
xxxxxxx 2
File caroleeg_sess3, length 1139, res 1.49
npz saved in data clean : focus, subjtime and labels 
599 599 603
(599, 2)


 44%|████▍     | 4/9 [00:00<00:00, 32.83it/s]

...

Best number of cluster is 3
xxxxxxx 3
File caroleeg_sess4_2, length 377, res 1.49
npz saved in data clean : focus, subjtime and labels 
528 528 532
(528, 2)


 22%|██▏       | 2/9 [00:00<00:00, 19.72it/s]

Best number of cluster is 7
xxxxxxx 4
File caroleeg_perf1, length 528, res 1.49
npz saved in data clean : focus, subjtime and labels 
701 701 706
(701, 2)


100%|██████████| 9/9 [00:00<00:00, 16.21it/s]


Best number of cluster is 4
xxxxxxx 4
File caroleeg_perf2, length 701, res 1.49
npz saved in data clean : focus, subjtime and labels 
